In [1]:
%matplotlib inline

import codecs
import spacy
import networkx as nx
import matplotlib.pyplot as plt
import csv
import n2w
import numpy as np
import matplotlib.pyplot as plt
import time
import nltk
import pickle
import os
import difflib
import bisect
import sys
import cvxpy
import scipy
import random
import re
import hnswlib
import string

from necessary_funcs import *
from sklearn.neighbors import NearestNeighbors

from nltk.corpus import stopwords 

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from datetime import datetime
from collections import Counter
from iteration_utilities import unique_everseen
from nltk.corpus import stopwords
from geotext import GeoText
from scipy.stats import norm
from rdflib import Graph
from networkx.readwrite import json_graph
from sklearn.metrics import jaccard_score
from os.path import expanduser
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
from scipy.special import factorial
from scipy.stats import expon
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from copy import deepcopy
from os import listdir
from os.path import isfile, join
from spacy.tokenizer import Tokenizer
from copy import deepcopy

C:\Users\nd\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Hyperparameters

Pay special attention to__:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;vectype: _(vals: 'glove','w2v'), type of vector you're using_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;sample_text_name: _name of the patent_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dloads: _full patent filename_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;bin_lower, bin_upper: _the bins within histogram of TFIDF values_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;n_fkdoc: _number of fake docs_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;threshold_f, mu: _reference the paper_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;lower, upper: _bounds on number of fake document, ref. the paper_  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;window_size: _size of the context window around the unknown word_  

In [2]:
DIST_OPTION = 'Euclid'

plt.switch_backend('agg')


sample_text_name = 'US20120245375A1'#random.sample(names, 1)[0] #'US2996519A'


dloads = './patents_html/' + sample_text_name + '_good.txt'

print('opening up patent ', dloads)

name = 'en'
# nlp = spacy.load('en')
nlp = spacy.load('en_core_web_sm')
         
bool_repeat = False#bool(sys.argv[0])

bin_lower = 2#binss[0] # 3
bin_upper = 18#binss[1] # 9


bin_range = (bin_lower - 1, bin_upper + 1)


n_fkdoc = 3#int(sys.argv[1])
threshold_f = 1.0# for explicit optim
threshold_r = 1. # for implicit optim

lower = 2
upper = 70
mu = 2
window_size = 10
vectype = 'w2v'

opening up patent  ./patents_html/US20120245375A1_good.txt


__Open the patent txt file__

In [3]:
with open(dloads, 'r') as f:
    labelText = f.read()

## Following cell loads variety of helper objects, their names implying their function



__weforge__: string; name of the folder which contains all of the helper files  
__cluster_to_vocab_index__: dict;_mapping from the cluster id to word vector vocabulary indices of words within the cluster (e.g. 2000 -> [1, 13, 52])_  
__inx_to_vectors__: dict; _vocab index to vector value (e.g. 99 -> [0.5 -0.32 ... 1])_  
__inx_to_word__: dict;_vocab index to a word (e.g. 101 -> 'backbone')_  
__centroids__: dict; numpy array of all centroid values  
__cluster_assignments__: list; _array cluster ids; each value is index by vocabulary position_  
__unsorted_vocab__: list; _regular, unsorted word vector vocabulary (keep in mind that GloVe and Word2Vec have different vocabs due to different tokenization schemes)_  
__sorted_vocab_indices__: list; _array of integers that represent sorted order of unsorted_vocab ( NOTE: we don't want to store both sorted and unsorted vocabulary, but we also don't want to sort the vocabulary at fake doc generation time, so this is a compromise solution)_  
__transform_matrix__: np.array; 300 x 300 matrix that maps context vector into regular word vector space  
__tfidf__: TfidfVectorizer object; learned on the corpus (by calling .fit(corpus)). Contains both idf vocabulary (since TfidfVectorizer tokenizes text itself, so it might differ from word vector vocabulary) and the idf values themselves. NOTE: you have to compute tf values yourself and then multiply them by idf values.  
__neighbors__: np.array; array of neighbors for each word, that is a 2d matrix of dimensionality (VOCAB_SIZE x NUM_NEIGHBORS)  

In [4]:
weforge = './weforge_new/'
# mapping:
# cluster_id --> [list of indexes of words in the cluster]
cluster_to_vocab_index = pload(weforge, vectype + '_cluster_to_indices')

# mapping:
# index --> [the word vector]

inx_to_vectors = pload(weforge, vectype +  '_vocab_inx_to_vector')
inx_to_word = pload(weforge, vectype + '_vocab_inx_to_word')

# LOAD DICTIONARY FROM WORD TO INDEX IN GLOVE
word_to_index = pload(weforge, vectype + '_word_to_inx')
# neighbors = pload(weforge, vectype + '_neighbors')
# ca - centroids
centroids = pload(weforge, vectype + '_ca')
# array:
# glove cluster_assignments [cluster_id_1 ... cluster_(num_unique_words)]
# inx                      [0            ...  num_unique_words]
cluster_assignments = pload(weforge, vectype + '_assignments')

# load sorted vocabulary to find insertion point for a word in the document


unsorted_vocab = pload(weforge, vectype + '_vocab')
sorted_vocab_indices = pload(weforge, vectype + '_sorted_vocab_indices')


sorted_vec_vocab = [unsorted_vocab[ii] for ii in sorted_vocab_indices]

transform_matrix = pload(weforge  + '/training_set/',  vectype + '_transform_matrix')


cluster_to_indices = pload(weforge, vectype + '_cluster_to_indices')

tfidf = pload(weforge, '/tfidf_object')

neighbors = pload(weforge, vectype + '_neighbors')

C:\Users\nd\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.0 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\nd\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.0 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


## Load actual vectors
Notice that we look at last the 300 values and not values from 1 to 301. That's because some words are composed of multiple words

In [5]:
if vectype == 'glove':
    vectors = []
    with open(weforge + '/glove_subset_vector.txt', 'r') as f:
        for ii, i in enumerate(tqdm(f)):
            z = i.strip().split()
            zz = [float(tok) for tok in z[-300:]]
            vectors.append(zz)

    vectors = np.array(vectors)
else:
    w2v = Word2Vec.load(weforge + '/model_subset.bin')
    vectors = w2v.wv.vectors # extract vectors from the model
    # word2index = {token: w2v.wv.vocab.get(token).index for token in w2v.wv.vocab}

## Process patent text: convert into the form we can work with
  
1. Break down entire patent into sequence of sentences.
2. Tag parts of speech within each sentence.  
3. Break down text into one long continuous sequence of tokens.    
&nbsp;&nbsp;&nbsp;&nbsp;_e.g. [['He', 'ran', 'away' '.'], ['It', 'was', 'unbearable', '.']] turns into ['He', 'ran', 'away', '.', 'It', 'was', 'unbearable' ,'.']_

In [6]:
# import nltk
# # nltk.download('punkt')
# nltk.download('stopwords')

In [7]:
##############################
#####
#####   TEST TEXT PROCESSING
#####
##############################
sentences = nltk.sent_tokenize(labelText)

word_tokens = []
for sent in sentences:
    
    x = nltk.pos_tag(nltk.word_tokenize(sent))
    word_tokens.extend(x)


execute = True

noun_loc = {}
noun_map = {}


remove = string.punctuation
remove = remove.replace("-", "") # don't remove hyphens
table = str.maketrans('', '', remove)
stop_words = set(stopwords.words('english') + ['thereof', 'wherein']) 

## Keep the nouns

Filter out every part of the speech except for the nouns. Then, lowercase them and remove all of the punctuation except hyphen

In [8]:
if execute:
    p_nouns = []
    for i in range(len(word_tokens)):
        if (word_tokens[i][1] == 'NN'): 
            if word_tokens[i][0] not in stop_words:
                p_nouns.append(word_tokens[i][0])

                if word_tokens[i][0] in noun_loc:
                    noun_loc[word_tokens[i][0]].append(i)
                else:
                    noun_loc[word_tokens[i][0]] = [i]
                    
    noun_list = []
    for i in p_nouns:

        p = deepcopy(i)

        i = i.strip() #remove leading and trailing spaces
        i = i.translate(table)

        if len(i) > 1:
            noun_list.append(i.lower())

            noun_map[i.lower()] = p

    # unique nouns in the list
    noun_set = set(noun_list)
    noun_set = list(noun_set)

else:
    if execute:
        with open(home + '/noun_set_' + name + '.pkl', 'rb') as f:
            noun_set = pickle.load(f)

            
            
            
full_noun_set = deepcopy(noun_set)            

## TF-IDF computation: step 1


1. Use CountVectorizer object to compute term frequencies
2. Find out the tokenization that CountVectorizer performed, i.e. load the vocabulary 
3. Match this vocabulary to the IDF vocabulary, as there might be words mispelled in the patent, or words not found in the original corpus

In [9]:
# Level 1: Count frequency of terms within document of interest
# Level 2: Pass document as a single text (has to be within a list) 
count = CountVectorizer()
label_text_as_document = [labelText]

counts = count.fit(label_text_as_document)
doc_vocab = counts.vocabulary_
sorted_doc_vocab = sorted(doc_vocab.keys())
term_counts = (count.fit_transform(label_text_as_document)).todense() # call dense to convert sparse matrix to normal numpy array


# np.array of shape (N, )
idf_vals = tfidf.idf_

idf_corpus_vocab = list(tfidf.vocabulary_)
tfidf_vocab_sorted_indices = pload(weforge, '/tfidf_vocab_sorted_indices')
tfidf_sorted_vocab = [idf_corpus_vocab[ii] for ii in tfidf_vocab_sorted_indices]

## TF-IDF computation: a digression

Before we proceed with actual computation of tf-idf values, we need to define a function that will filter out nouns that have low IDF values in a random corpus. Why? By definition low IDF implies high document frequency, which in turn implies that these words are representative of this random corpus. Thus, these words' discriminative value for our metallurgy corpus is low and we need to remove them from consideration.

In [10]:
# random_tfidf = pload(weforge, '/random_tfidf_object')
# random_idf_vals = random_tfidf.idf_

# random_idf_corpus_vocab = list(random_tfidf.vocabulary_)
# random_tfidf_vocab_sorted_indices = pload(weforge, '/random_tfidf_vocab_sorted_indices')
# random_tfidf_sorted_vocab = [random_idf_corpus_vocab[ii] for ii in random_tfidf_vocab_sorted_indices]

# entries, bin_middles, bin_edges = hgram(random_idf_vals)



# random_left_edge = bin_edges[7]
# random_right_edge = bin_edges[-1]
def find_nonmatching_words_for_list(noun_list, sorted_vocab):
    l = []
    matches = {}
    for noun in noun_list:
        loc = find_insertion_point(noun, sorted_vocab)
        left_bound = bound_aware_op(loc - 1, 0, len(sorted_vocab))
        right_bound = bound_aware_op(loc + 1, 0, len(sorted_vocab))
        matches[noun] = difflib.get_close_matches(noun, [sorted_vocab[left_bound], sorted_vocab[right_bound]], cutoff=0.8)
        if len(matches[noun]) == 0:
            l.append(noun)
    return l


print('before', len(noun_set), noun_set)

bad_words = pload(weforge, '/random_tfidf_sorted_words_2')
noun_set = [x for x in noun_set if x not in bad_words]

noun_set = find_nonmatching_words_for_list(noun_set, bad_words)



print('after', len(noun_set),noun_set)


before 190 ['wt', 'performance', 'increase', 'carbon', 'stirring', 'example', 'process', 'completion', 'intermediate', 'meterability', 'need', 'olefin', 'addition', 'decomposition', 'feed', 'wash', 'preference', 'proportion', 'temperature', 'aldehyde', 'analysis', 'protonation', 'risk', 'monoester', 'equivalent', 'oxide', 'derivative', 'cycloaddition', 'method', 'ozonide', 'use', 'potassium', 'position', 'trickle', 'oxygen', 'group', 'core', 'processing', 'exchange', 'context', 'advantage', 'problem', 'object', 'instance', 'mlmin', 'embodiment', 'oxidizing', 'discharge', 'system', 'manner', 'chain', 'removal', 'field', 'azelate', 'production', 'priority', 'effect', 'news', 'turn', 'peroxide', 'ester', 'water', 'conversion', 'comprising', 'solvent', 'reduction', 'air', 'avoidance', 'batchwise', 'condenser', 'substrate', 'dioxide', 'formation', 'conc', 'sulphide', 'heat', 'react', 'alkyl', 'concentration', 'oleate', 'option', 'aspect', 'scheme', 'safer', 'reactor', 'inlet', 'hydrogenium'

## TF-IDF computation: step 2

Next step in computing tf-idf is figuring out what words in the patent of interest are present in the corpus vocabulary found by TfIdfVectorizer(). Likely, we won't find exact matches, so we settle for the ones that are above certain threshold (Levenstein distance of 0.6).


Similarly, find matches within CountVectorizer tokenized vocab.

In [11]:
##############################
#####
#####   FIND MATCHES OF NOUNS IN THE TEST DOCUMENT
#####   IN COUNT VECTORIZER TOKENIZED TEXT
#####   IN IDF TOKENIZED CORPUS
#####
##############################

from collections import Counter

tfidf_matches = {}
idf_matches = {}

print(Counter(noun_set))
for noun in tqdm(noun_set):
    tfidf_matches[noun] = find_matching_word(noun, tfidf_sorted_vocab)
    idf_matches[noun] = find_matching_word(noun, sorted_doc_vocab)
    
#     print(noun, tfidf_matches[noun])
#     print(noun, idf_matches[noun])




tfidf_vals = []
indices_that_survive = []
unknown = []

100%|██████████| 69/69 [00:00<00:00, 17249.19it/s]

Counter({'process': 1, 'meterability': 1, 'olefin': 1, 'feed': 1, 'protonation': 1, 'risk': 1, 'monoester': 1, 'derivative': 1, 'cycloaddition': 1, 'use': 1, 'potassium': 1, 'oxygen': 1, 'core': 1, 'processing': 1, 'context': 1, 'mlmin': 1, 'embodiment': 1, 'oxidizing': 1, 'manner': 1, 'chain': 1, 'removal': 1, 'priority': 1, 'turn': 1, 'ester': 1, 'water': 1, 'conversion': 1, 'avoidance': 1, 'batchwise': 1, 'dioxide': 1, 'react': 1, 'oleate': 1, 'scheme': 1, 'safer': 1, 'reactor': 1, 'pka': 1, 'countercurrent': 1, 'order': 1, 'alpha': 1, 'oil': 1, 'form': 1, 'fragment': 1, 'acceleration': 1, 'volume': 1, 'reaction': 1, 'acetone': 1, 'adduct': 1, 'flask': 1, 'decarboxylation': 1, 'entirety': 1, 'degradation': 1, 'weight': 1, 'recombine': 1, 'offgas': 1, 'succession': 1, 'apparatus': 1, 'summary': 1, 'hydroperoxide': 1, 'section': 1, 'triphenylphosphine': 1, 'time': 1, 'content': 1, 'percent': 1, 'stream': 1, 'isopropanol': 1, 'pure': 1, 'ozonolysis': 1, 'monomethyl': 1, 'carboxyl': 1, 

## TF-IDF computation: step 3

Finally, compute tf-idf based on two following factors:
1. Whether we found a match in idf vocab
2. Whether we found a match in CountVectorizer vocab

If a word fails one of these conditions we remove it from consideration.

Once we computed tf-idf values we call a function called hgram to bin values. Once we do that we can filter out words whose tf-idf value is the range outside of the one specified by the hyperparams: bin_lower, bin_upper


In [12]:
##############################
#####
#####   COMPUTE TF-IDF
#####
##############################



for i, noun in enumerate(tqdm(noun_set)):
    if len(idf_matches[noun]) > 0 and len(tfidf_matches[noun]) > 0:
        tf_index = doc_vocab[idf_matches[noun][0]]
        tf_val = term_counts[0,tf_index]/np.sum(term_counts)

        idf_vocab_index = tfidf.vocabulary_[tfidf_matches[noun][0]]
        idf_val = idf_vals[idf_vocab_index]

        tfidf_vals.append(tf_val*idf_val)

        indices_that_survive.append(i)
    else:
        if len(noun) > 2:
            unknown.append(noun)





tfidf_vals = np.array(tfidf_vals)
entries, bin_middles, bin_edges = hgram(tfidf_vals)

# BIN HYPERPARAMS


left_edge = bin_edges[bin_range[0]]
right_edge = bin_edges[bin_range[1]]


first_left_edge = bin_edges[0]
first_right_edge = bin_edges[1]



print('bin_edges', bin_edges)
print('indices_that_survive', len(indices_that_survive), 'tfidf_vals', len(tfidf_vals))
noun_set = [noun_set[i] for i in indices_that_survive]
print('[noun_set[i] for i in indices_that_survive]', noun_set)

first_bin = [noun for i, noun in enumerate(noun_set) if tfidf_vals[i] >= first_left_edge and tfidf_vals[i] <= first_right_edge] 
first_bin_tfidf_vals = [tfidf_vals[i] for i, noun in enumerate(noun_set) if tfidf_vals[i] >= first_left_edge and tfidf_vals[i] <= first_right_edge]
plt.clf()

print('first_bin', first_bin, 'first_bin_tfidf_vals', first_bin_tfidf_vals)

first_entries, first_bin_middles, first_bin_edges = hgram(first_bin_tfidf_vals)

fbe_len = len(first_bin_edges)

fbe_first = first_bin_edges[fbe_len - 6]
fbe_last = first_bin_edges[fbe_len - 1]


plt.show()


noun_set = list(noun_set)


# Remove the words that do not have TFIDF values in proper range
# Remove the words that do not have a match in corpus vocab
# noun_set = [noun_set[i] for i in indices_that_survive]

noun_set = [(noun,tfidf_vals[i])  for i, noun in enumerate(noun_set) if tfidf_vals[i] >= left_edge and tfidf_vals[i] <= right_edge] 
#temp = [tfidf_vals[i] for i, noun in enumerate(noun_set) if tfidf_vals[i] >= left_edge and tfidf_vals[i] <= right_edge]

cpy = deepcopy(noun_set)


noun_set = [x[0] for x in cpy]
tfidf_vals =[x[1] for x in cpy]
# add_noun_set = [noun for i, noun in enumerate(first_bin) if first_bin_tfidf_vals[i] >= fbe_first and first_bin_tfidf_vals[i] <= fbe_last]

# print('old noun_set  ', noun_set,add_noun_set)

# noun_set = noun_set + add_noun_set

# tfidf_vals = tfidf_vals + first_bin_tfidf_vals

# print(' noun_set  ', noun_set)

##############################
#####
#####   
#####
##############################



print('tfidf_vals', noun_set)

print('tfidf_vals', tfidf_vals)
# once you found where the word of interest lies, use 0.6 levenstein distance cutoff to figure out if 
# there is a close match
matches = find_matching_words_for_list(noun_set, sorted_vec_vocab)

100%|██████████| 69/69 [00:00<00:00, 71124.84it/s]

bin_edges [0.0015077  0.00680859 0.01210947 0.01741036 0.02271125 0.02801213
 0.03331302 0.03861391 0.04391479 0.04921568 0.05451657 0.05981745
 0.06511834 0.07041923 0.07572011 0.081021   0.08632189 0.09162277
 0.09692366 0.10222454 0.10752543]
indices_that_survive 68 tfidf_vals 68
[noun_set[i] for i in indices_that_survive] ['process', 'meterability', 'olefin', 'feed', 'protonation', 'risk', 'monoester', 'derivative', 'cycloaddition', 'use', 'potassium', 'oxygen', 'core', 'processing', 'context', 'embodiment', 'oxidizing', 'manner', 'chain', 'removal', 'priority', 'turn', 'ester', 'water', 'conversion', 'avoidance', 'batchwise', 'dioxide', 'react', 'oleate', 'scheme', 'safer', 'reactor', 'pka', 'countercurrent', 'order', 'alpha', 'oil', 'form', 'fragment', 'acceleration', 'volume', 'reaction', 'acetone', 'adduct', 'flask', 'decarboxylation', 'entirety', 'degradation', 'weight', 'recombine', 'offgas', 'succession', 'apparatus', 'summary', 'hydroperoxide', 'section', 'triphenylphosphin


<ipython-input-12-e0bd8f5b26e4>:62: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [13]:
# ##############################
# #####
# #####   KNOWN WORDS PROCESSING:
# #####   1. 
# #####   2.
# ##### 
# #####
# ##############################
clusters_of_interest = []
cluster_to_noun = {}
nn_dist = {}
nn_word = {}



max_seq_len = 0
noun_to_util_index = {}
lens = []


# # ADD
# # BIN FILTERING 
nn_vals = []
noun_counter = 0 
indices_that_survive = []
r_a = {}
noun_to_neighbors = {}
nouns_that_survive = {}
counter = 0

nn_dist_implicit = {}
noun_to_neighbors_implicit = {}
nouns_that_survive_implicit = {}
nn_vals_implicit = []
counter_implicit = 0
noun_counter_implicit = 0
max_seq_len_implicit = 0
lens_implicit = []
indices_that_survive_implicit = []

### Loop which goes thru relevant nouns (the ones that survived tf-idf bin filtering) and finds their nearest neighbors (removes the ones with the same stem as the noun to be replaced) and computes the NN distances (because K-Means/K-NN code doesn't provide distances)

In [14]:
from nltk.stem.porter import *

stemmer = PorterStemmer()

# print(unknown_word_neighbors)
    
    
# word_to_index
# cluster_assignments
# cluster_to_vocab_index
# neighbors

indices_that_survive = []
# FOR EVERY RELEVANT NOUN WE FIND THE NEIGHBORS THAT ARE NOT IN THE DOCUMENT AND WE ALSO COMPUTE DISTANCE BETWEEN THIS WORD AND EVERY NEIGHBOR
for i, noun in enumerate(tqdm(noun_set)):
    #nn_vals.append([])
    if len(matches[noun]) > 0:
        nn_dist[noun] = []
         # 2d list that will be turned to costs matrix using numpy 
        noun_to_util_index[noun] = i # dic from noun to its index in the matrix that will be used in optimization
        word_index = word_to_index[matches[noun][0]]
        cluster_id = cluster_assignments[word_index]
        clusters_of_interest.append(cluster_id)

        # if statement that exists only to fill in 
        # the cluster_to_noun 
        if cluster_id not in cluster_to_noun:
            cluster_to_noun[cluster_id] = [noun]
        else:
            cluster_to_noun[cluster_id].append(noun)

        # confusing naming: this step returns all other members of the cluster current word 'noun'
        # belongs to 
        other_members = cluster_to_vocab_index[cluster_id]
        ngbrs = neighbors[word_index]
        ngbrs = [xx for xx in ngbrs if xx != word_index]
        

        # BUG: ordering(distances) are not preserved by the set operations 
        x_implicit = set(ngbrs).intersection(set(other_members)) # set of all nearest neighbors who are also in the cluster
        x_implicit = x_implicit.difference(noun_set)
        
        x = set(ngbrs)
        x = x.difference(noun_set) # set of all nearest neighbors who are not in the document 
        
        neighbor_order = {}
        neighbor_order_implicit = {}
        
        
        num_neighbors_for_cur_noun = len(x)
        num_neighbors_for_cur_noun_implicit = len(x_implicit)
        
        ###########
        # EXPLICIT COMPUTATION
        ###########

        # set difference might come up empty
        if num_neighbors_for_cur_noun > 0:
            nouns_that_survive[counter] = noun
            nn_vals.append([])

            if num_neighbors_for_cur_noun > max_seq_len:
                max_seq_len = num_neighbors_for_cur_noun
                
            # for each word in a neighbor set 
            # find its ordering within older list of neighbors (as a proxy for proximity)
            nn_dist[noun] = []
            dist_run_sum = 0
            order_of_indices = []
            noun_to_neighbors[noun] = []
            ngbrs = []

            
            for j, neighbor in enumerate(x):
                if word_index != neighbor and stemmer.stem(noun) != stemmer.stem(inx_to_word[neighbor]):
                    neighbor_order[neighbor] = np.where(ngbrs == neighbor)[0]
                    order_of_indices.append(neighbor_order[neighbor])
                    ngbrs.append(neighbor)


                    # compute L2 dist between the noun of interest and its neighbor
                    # HYPERPARAM 'ord', for Manhattan 1, Euclidian 2
                    if DIST_OPTION == 'Manhattan' or DIST_OPTION == 'Euclid':
                        dist = np.linalg.norm(np.array(inx_to_vectors[word_index]) -  np.array(inx_to_vectors[neighbor]), ord=2)
                    else:
                        dist  = scipy.spatial.distance.cosine(np.array(inx_to_vectors[word_index]), np.array(inx_to_vectors[neighbor]))

                    # nn_dist[noun].append((word_to_index[neighbor], neighbor_order[neighbor], dist))
                    nn_vals[noun_counter].append(dist)
                    dist_run_sum += dist
                else:
                    num_neighbors_for_cur_noun = num_neighbors_for_cur_noun - 1
        
            
            noun_to_neighbors[noun] = ngbrs

            lens.append(num_neighbors_for_cur_noun)
            noun_counter += 1
            indices_that_survive.append(i)
            
            counter += 1
            
        ###########
        # IMPLICIT COMPUTATION
        ###########
        if num_neighbors_for_cur_noun_implicit > 0:
    
            nouns_that_survive_implicit[counter] = noun
            nn_vals_implicit.append([])

            if num_neighbors_for_cur_noun_implicit > max_seq_len_implicit:
                max_seq_len_implicit = num_neighbors_for_cur_noun_implicit
                

            
            nn_dist_implicit[noun] = []
            dist_run_sum_implicit = 10e-42
            order_of_indices_implicit = []
            noun_to_neighbors_implicit[noun] = []
            ngbrs_implicit = []

            for j, neighbor_implicit in enumerate(x_implicit):
                print(noun, inx_to_word[neighbor_implicit])
                print(word_index != neighbor_implicit, stemmer.stem(noun) != stemmer.stem(inx_to_word[neighbor_implicit]))
                
                

                if word_index != neighbor_implicit and stemmer.stem(noun) != stemmer.stem(inx_to_word[neighbor_implicit]):
                    neighbor_order_implicit[neighbor_implicit] = np.where(ngbrs_implicit == neighbor_implicit)[0]
                    order_of_indices_implicit.append(neighbor_order_implicit[neighbor_implicit])
                    ngbrs_implicit.append(neighbor_implicit)

                    # compute L2 dist between the noun of interest and its neighbor
                    # HYPERPARAM 'ord', for Manhattan 1, Euclidian 2
                    if DIST_OPTION == 'Manhattan' or DIST_OPTION == 'Euclid':
                        dist_implicit = np.linalg.norm(np.array(inx_to_vectors[word_index]) -  np.array(inx_to_vectors[neighbor_implicit]), ord=2)
                    else:
                        dist_implicit  = scipy.spatial.distance.cosine(np.array(inx_to_vectors[word_index]), np.array(inx_to_vectors[neighbor_implicit]))

                    # nn_dist[noun].append((word_to_index[neighbor], neighbor_order[neighbor], dist))
                    nn_vals_implicit[noun_counter_implicit].append(dist_implicit)
                    
                    dist_run_sum_implicit += dist_implicit
                else:
                    num_neighbors_for_cur_noun_implicit = num_neighbors_for_cur_noun_implicit - 1
    

#             r_a[noun] = (len(xx)**2)/dist_run_sum_implicit
            r_a[noun] = (len(noun_to_neighbors_implicit)**2)/dist_run_sum_implicit
            noun_to_neighbors_implicit[noun] = ngbrs_implicit
            lens_implicit.append(num_neighbors_for_cur_noun_implicit)
            indices_that_survive_implicit.append(i)
            noun_counter_implicit += 1
            

            counter_implicit += 1

  0%|          | 0/18 [00:00<?, ?it/s]

process method
True True
derivative cycloheteroalkyl
True True
cycloaddition aminoketone
True True
cycloaddition 5-amino-34-dihydro-1
True True
cycloaddition 5-nitro-34-dihydro-1
True True
cycloaddition ring-closure
True True
cycloaddition aromatisation
True True
cycloaddition -naphthylimine
True True
cycloaddition -naphthalenone
True True
use utilize
True True
use employ
True True
use us
True True
embodiment aspect
True True
oxidizing absorbs
True True
oxidizing oxidise
True True
water bittern
True True
water seawater
True True
water saline
True True
oleate 2-ethyl
True True
oleate xanthate
True True
oleate carboxymethylcellulose
True True
oleate carboxymethyl
True True


100%|██████████| 18/18 [00:00<00:00, 130.44it/s]

alpha beta
True True
offgas off-gases
True True
offgas offgases
True True


# Explicit optimization

In [15]:
# Fast way to create costs mask 
# print(lens)
maxlen = max(lens)
costs = np.zeros((len(nn_vals),maxlen),int)
mask = np.arange(maxlen) < np.array(lens)[:,None] # key line
costs[mask] = np.concatenate(nn_vals)    # fast 1d assignment

tfidf_vals = np.array(tfidf_vals)
relevant_tf_idf_vals = tfidf_vals[indices_that_survive]
utilities = np.diag(relevant_tf_idf_vals) # tf-idf values or \xi sign in the document

### Optimization equation

$$\min \sum\limits_{f\in\mathcal{F}}\sum\limits_{c\in d}\sum\limits_{c'\in \mathcal{FC}(c,d)} ||v_c-v_{c'}||\cdot \xi_c\cdot X_{c,c',f}$$
$$$$
$$\text{subject to}\,\,\, (i) \,\,\,  \sum\limits_{f\in \mathcal{F}}\sum\limits_{c'\in \mathcal{FC}(c,d)}X_{c,c',f}\leq \mu, \forall c\in d$$
$$$$
$$(ii) \,\,\, \sum\limits_{c'\in \mathcal{FC}(c,d)}X_{c,c',f}\leq 1, \forall c\in d,f\in \mathcal{F}$$
$$$$
$$(iii) \,\,\, N_l\leq \sum\limits_{c\in d}\sum\limits_{c'\in \mathcal{FC}(c,d)}X_{c,c',f} \leq N_u, \ \forall f\in \mathcal{F}$$
$$$$
$$(iv) \,\,\, \sum\limits_{c\in d}\sum\limits_{c'\in\mathcal{FC}(c,d)}||v_c-v_{c'}||X_{c,c',f}\geq T_f, \ \forall f\in \mathcal{F}$$
$$$$
$$(v)\,\,\, X_{c,c',f}=0,\ \forall c\notin \mathcal{B}$$
$$$$
$$(vi)\,\,\, X_{c,c',f} \in \{0,1\}, \ \forall c,c',f$$



$X_{c,c',f}\in\{0,1\}$ is a binary variable, where $X_{c,c',f}$=1 means a concept $c\in d$ will be replaced by a candidate $c'\in\mathcal{FC}(c,d)$ to generate a fake document $f\in\mathcal{F}$. We explicitly encode the concept replacement in the joint optimization problem. Constraint (i) indicates that a concept can be replaced at most $\mu$ times. Constraint (ii) restricts that the number of concepts that are going to be replaced in each fake documentris bounded by $N_l$ and $N_u$. Constraint (iii) means that for each fake document $f \in \mathcal{F}$, the overall replacement availability should not be smaller than a threshold $T_f$ (so that it will be able to be notably different from the original document to disguise important information). Constraint (iv) is such that the overall distance of concepts and their replacements is constrained to be equal or larger than a customer-specified threshold $T_f$. In addition, we add another constraint (ii) to enforce that each fake document $f$ and each concept $c$, at most one of the concepts in the feasible candidate set $\mathcal{FC}(c,d)$ is used to replace the target concept $c$.


In [16]:
!pip install cvxopt
import cvxopt

In [17]:
global list_of_concepts
list_of_concepts = []



num_concepts = len(nn_vals)#len(outputs)
full_selection = {}
for i in range(n_fkdoc):
    full_selection[i] = cvxpy.Variable((num_concepts, max_seq_len), boolean=True)


ones = np.ones((1, max_seq_len))

replacement_constraint = []


num_concepts_to_be_replaced_constraint = []
dist_constraint = []

for i in range(n_fkdoc):
    replacement_constraint.append(cvxpy.sum(full_selection[i], 1) <= 1)
    dist_constraint.append(cvxpy.sum(cvxpy.multiply(full_selection[i], costs))  >= threshold_f)
    num_concepts_to_be_replaced_constraint.append(cvxpy.sum(full_selection[i]) <= upper)
    num_concepts_to_be_replaced_constraint.append(lower <=  cvxpy.sum(full_selection[i]))

    #num_times_you_can_replace_concept_across_fake_docs = num_times_you_can_replace_concept_across_fake_docs #+  full_selection[i]


num_times_you_can_replace_concept_across_fake_docs = full_selection[0]
for i in range(1, n_fkdoc):
    num_times_you_can_replace_concept_across_fake_docs += full_selection[i]


num_times_you_can_replace_concept_across_fake_docs = cvxpy.sum(num_times_you_can_replace_concept_across_fake_docs, 1)
mu_cstrt = np.ones((len(nn_vals)))*mu
nconstraint = [num_times_you_can_replace_concept_across_fake_docs <= mu]

total_utility = 0
for i in range(n_fkdoc):
    # (N x N) x (N x max_seq_len) =  (N x max_seq_len)
    weighted_costs = utilities.dot(costs)

    #  (max_seq_len x N)  x (N x max_seq_len) x (max_seq_len x 1)
    total_utility += cvxpy.sum(cvxpy.multiply(full_selection[i],weighted_costs))




all_constraints = replacement_constraint  + num_concepts_to_be_replaced_constraint + dist_constraint + nconstraint

knapsack_problem = cvxpy.Problem(cvxpy.Maximize(total_utility),  all_constraints)

# knapsack_problem.solve()


# Generate fake docs thru explicit optimization done above

In [18]:
fake_docs_explicit = []

lst_target = []

lst_sub = []


if all([full_selection[i].value is not None for i in range(n_fkdoc)]):

    for i in range(n_fkdoc):
        xx = np.round(full_selection[i].value)
        print()

        text1 = deepcopy(labelText)


        print('FOR a FAKE DOC, EXPLICIT optimization', i+1)
        for j in range(xx.shape[0]):
            if len(np.nonzero(xx[j,:] > 0.9)[0]) > 0: 
                index = np.nonzero(xx[j,:] > 0.9)[0][0]

                noun = nouns_that_survive[j]
                list_of_neigbrs = noun_to_neighbors[noun]


                replacement_index = list_of_neigbrs[index]

                replacement = re.sub(r'\W+', '', inx_to_word[replacement_index])
                


                print('replace', noun,'with', replacement)



                target = noun



                tmp1 = re.search('\b' + target + '\b', text1)
                if tmp1 !="":
                    if(target[0].isupper()):
                        replacement = replacement.capitalize()
                    
                    text1 = re.sub(r"\b%s\b" % target , replacement, text1)
                    lst_target.append(target)
                    lst_sub.append(replacement)
                    
                if(target[0].isupper() and not target[1].isupper() ):
                        target = target.lower()  
                        replacement = replacement.lower()
                        if target in text1:
                            text1 = re.sub(r"\b%s\b" % target , replacement, text1)

                            lst_target.append(target)
                            lst_sub.append(replacement)
                        
        if len(lst_target) > 0:
            fake_docs_explicit.append(text1)

    listt = 'explicit_fake_doc_no_' + str(i) + '_' + sample_text_name + '_blbulu_' + str(bin_lower)  + 'd' + str(bin_upper) + 'd' + str(lower)  + 'd' + str(upper) +  'w' + '_listtt'
    
    print('home', home)
    print('listt', listt)
    with open('./' + listt, 'w') as f:
        for target, replacement in zip(lst_target, lst_sub):
            f.write('replace' + ' ' + target + ' with ' + replacement + '\n')



    for i, text in enumerate(fake_docs_explicit):
        with open('./explicit_fake_doc_no_' + str(i) + '_' + sample_text_name + '_blbulu_' + str(bin_lower)  + 'd' + str(bin_upper) + 'd' + str(lower)  + 'd' + str(upper), 'w') as f:
            f.write(text)
else:
    print('Didn\'t find a solution for explicit optimization')

Didn't find a solution for explicit optimization


## Set up variables for implicit optimization

In [19]:
num_concepts = len(nn_vals_implicit)#len(outputs)
tfidf_vals = np.array(tfidf_vals)
# CREATE COSTS MATRIX 
# THIS CODE IS CONFUSING BUT FAST FOR LARGE MATRIX VALUES
# WHY? 
maxlen_implicit = max(lens_implicit)
costs_implicit = np.zeros((len(nn_vals_implicit),maxlen_implicit),int)
mask_implicit = np.arange(maxlen_implicit) < np.array(lens_implicit)[:,None] # key line
costs_implicit[mask_implicit] = np.concatenate(nn_vals_implicit)    # fast 1d assignment

relevant_tf_idf_vals_implicit = tfidf_vals[indices_that_survive_implicit]
utilities_implicit = np.diag(relevant_tf_idf_vals_implicit) # tf-idf values or \xi sign in the document


## IMPLICIT OPTIMIZATION

$$\min \sum\limits_{f\in\mathcal{F}}\sum\limits_{c\in d}ra(c,d)\cdot \xi_c\cdot X_{c,f}$$
$$$$
$$\text{subject to}\,\,\,(i)\,\,\,\sum\limits_{f\in \mathcal{F}}X_{c,f}\leq \mu, \forall c\in d$$
$$$$
$$(ii)\,\,\, N_l\leq \sum\limits_{c\in d}X_{c,f} \leq N_u, \ \forall f\in \mathcal{F}$$
$$$$
$$(iii)\,\,\, \sum\limits_{c\in d}ra(c,d)X_{c,f}\geq T_f, \ \forall f\in \mathcal{F}$$
$$$$
$$(iv)\,\,\, X_{c,f}=0, \forall c\notin \mathcal{B}$$
$$$$
$$\,\,\,(v)\,\,\, X_{c,f}\in \{0,1\}, \forall c,f$$


In [20]:
implicit_selection_variable = cvxpy.Variable((num_concepts, n_fkdoc), boolean=True)

implicit_num_concepts_for_all_fake_docs_constraint = [cvxpy.sum(implicit_selection_variable, 1) <= mu]

implicit_num_concepts_per_fake_docs_constraint = [cvxpy.sum(implicit_selection_variable, 0) >= lower, cvxpy.sum(implicit_selection_variable, 0) <= upper]



implicit_ra_multiplier = np.array(list(r_a.values()))

implicit_ra_multiplier = np.reshape(implicit_ra_multiplier, (num_concepts, 1))

implicit_ra_multiplier = np.tile(implicit_ra_multiplier, (1, n_fkdoc))

implicit_ra_constraint = [cvxpy.sum(cvxpy.multiply(implicit_selection_variable, implicit_ra_multiplier), 0) >= threshold_r]



all_implicit_constraints = implicit_num_concepts_per_fake_docs_constraint + implicit_num_concepts_for_all_fake_docs_constraint + implicit_ra_constraint


implicit_utility = cvxpy.sum(cvxpy.multiply(implicit_selection_variable, np.dot(utilities_implicit, implicit_ra_multiplier)))
implicit_knapsack_problem = cvxpy.Problem(cvxpy.Maximize(implicit_utility),  all_implicit_constraints)

implicit_knapsack_problem.solve()

1.1259994335290133

# Generate fake docs using implicit optimization

In [23]:
lst_target = []
lst_sub = []



fake_docs_implicit = []




if implicit_selection_variable.value is not None:
    implicit_fk_concepts_val = np.round(implicit_selection_variable.value)


    for i in range(n_fkdoc):

        text1 = deepcopy(labelText)

        xx = implicit_fk_concepts_val[:,i]
        print('FOR a FAKE DOC, implicit optimization', i+1)

        if len(np.nonzero(xx > 0.9)[0]) > 0: 
            indices = np.nonzero(xx > 0.9)[0]
            for inx in indices:
                noun = nouns_that_survive[inx]


                list_of_neigbrs = noun_to_neighbors[noun]

                index = random.randrange(0, len(list_of_neigbrs))

                replacement_index = list_of_neigbrs[index]

                replacement = re.sub(r'\W+', '', inx_to_word[replacement_index])

                target = noun

                tmp1 = re.search(r'\b' + target + r'\b', text1)
                if tmp1 !="":
                    if(target[0].isupper()):
                        replacement = replacement.capitalize()
                    
                    text1 = re.sub(r"\b%s\b" % target , replacement, text1)
                    lst_target.append(target)
                    lst_sub.append(replacement)
                    
                if (target[0].isupper() and not target[1].isupper() ):
                        target = target.lower()  
                        replacement = replacement.lower()
                        if target in text1:
                            text1 = re.sub(r"\b%s\b" % target , replacement, text1)

                            lst_target.append(noun)
#                             lst_target.append(target)
                            lst_sub.append(replacement)
                        
                print('replace', noun,'with', replacement)

                
                
        if len(lst_target) > 0:
            fake_docs_implicit.append(text1)


    listt = 'implicit_fake_doc_no_' + str(i) + '_' + sample_text_name + '_blbulu_' + str(bin_lower)  + 'd' + str(bin_upper) + 'd' + str(lower)  + 'd' + str(upper) + 'w' + '_listtt'


    with open('./' + listt, 'w') as f:
        for target, replacement in zip(lst_target, lst_sub):
            f.write('replace' + ' ' + target + ' with ' + replacement + '\n')




    for i, text in enumerate(fake_docs_implicit):
        with open('./' + 'implicit_fake_doc_no_' + str(i) + '_' + sample_text_name + '_blbulu_' + str(bin_lower)  + 'd' + str(bin_upper) + 'd' + str(lower)  + 'd' + str(upper), 'w') as f:
            f.write(text)

else:

    print('Didn\'t find a solution for implicit optimization')

FOR a FAKE DOC, implicit optimization 1
replace process with methanold4
replace cycloaddition with cyclotriacontane
replace use with emulsifier
replace oxygen with peroxocarbonate
replace embodiment with 2carboxybenzaldehyde
replace oxidizing with withdraws
replace ester with decarboxylation
replace water with wastewater
replace oleate with hexanoate
FOR a FAKE DOC, implicit optimization 2
replace process with tetrafluorosilane
replace derivative with multicoordination
FOR a FAKE DOC, implicit optimization 3
replace derivative with xny
replace cycloaddition with dimerisation
replace use with purchase
replace oxygen with superoxide
replace embodiment with eubacterium
replace oxidizing with 3chloro2hydroxypropyl
replace ester with octyl
replace water with antiscalant
replace oleate with 4hydroxybenzoate
